In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import numpy as np
import os
import time
import random
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import pickle
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

import pickle

seed =1
torch.manual_seed(seed)


from torchvision import transforms, utils
import torchvision.models as models
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

from multi_center_tools.multi_tools import open_pickle, select_df,load_TCGA_arrays,translate_mask, imshow
from multi_center_tools.ViT_tools import get_ViT_series,get_center_indexes,VitTCGAToTensor, Vit_TCGA_Dataset
import copy
import copy
from torchmetrics import F1Score
from sklearn.model_selection import KFold
f1ig0 = F1Score(num_classes=2, average='macro', ignore_index=0)
f1ig1 = F1Score(num_classes=2, average='macro', ignore_index=1)
import timm

2023-05-21 14:37:39.366267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 14:37:50.537215: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-21 14:38:21.162260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /uge_mnt/home/satoc/.raiden/nvcr-pytorch-2210/lib/python3.8/site-packages/cv2/../../lib64:/uge_mnt/home/satoc/.raiden/nvcr-pytorch-2210/lib:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/

In [2]:
csv_loc = '/home/satoc/vit_experiment/label_data/TCGA_training_nondu.csv'
img_loc = '/home/satoc/arrays/kino_scaling/FR_BraTS2019_array'

train_val_list = open_pickle(
    '/home/satoc/HvsAI/label_data/TCGA_list1_train.pickle')

test_list = open_pickle(
    '/home/satoc/HvsAI/label_data/TCGA_list1_test.pickle')

df =pd.read_csv(csv_loc)
train_val_pd = select_df(df,'BraTS_2019_subject_ID',train_val_list )
test_pd = select_df(df,'BraTS_2019_subject_ID',test_list )

#reset index
train_val_pd =train_val_pd.reset_index()
test_pd = test_pd.reset_index()


Z_score = False
SZ = True
Double = False
inst = None

Batch_size = 32
weights = torch.tensor([1.0, 1.0]).to(device)
Epoch = 20
lr = 3e-5
fold = 0

mask = False
channel = 4

save_name = 'TCGA10fold_fold' + str(fold)
print(save_name)

# Path to the folder where the pretrain_valed models are saved
CHECKPOINT_PATH = "./saved_models/" + save_name
if not os.path.exists(CHECKPOINT_PATH):
    os.makedirs(CHECKPOINT_PATH)

TCGA10fold_fold0


In [3]:
kf = KFold(n_splits=10, shuffle=True, random_state=0)
train_indexs=[]
val_indexs = []
for train_index, val_index in kf.split(train_val_pd):
    print(train_index)
    print(val_index)
    train_indexs.append(train_index)
    val_indexs.append(val_index)

[  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  23  24  25  27  28  29  30  31  32  34  35  36  37  38  39  40  41
  42  43  44  45  46  47  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  67  68  69  70  71  72  73  74  75  76  77  79  80
  81  82  83  84  85  86  87  88  89  92  93  94  95  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111]
[ 2  7 10 22 26 33 48 66 78 90 91 96]
[  0   1   2   3   4   5   6   7   9  10  11  12  14  15  17  18  19  20
  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  44  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  76  77
  78  79  80  81  83  84  85  86  87  88  90  91  94  95  96  97  98  99
 101 102 103 104 105 107 108 109 110 111]
[  8  13  16  24  45  75  82  89  92  93 100 106]
[  0   1   2   4   5   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25 

In [4]:
train_pd = train_val_pd.iloc[train_indexs[fold]]
val_pd = train_val_pd.iloc[val_indexs[fold]]

#delete level 0
train_pd = train_pd.iloc[:, 3:]
val_pd = val_pd.iloc[:, 3:]

train_pd = train_pd.reset_index()
val_pd = val_pd.reset_index()

In [ ]:
train_pd['tumor_index'] = get_center_indexes(train_pd, img_loc)
v_train_pd = pd.DataFrame(get_ViT_series(train_pd))
v_train_pd.reset_index()

val_pd['tumor_index'] = get_center_indexes(val_pd, img_loc)
v_val_pd = pd.DataFrame(get_ViT_series(val_pd))
v_val_pd.reset_index()

In [6]:
s_train = Vit_TCGA_Dataset(v_train_pd, img_loc, z_score=Z_score,sz=SZ, inclued_mask=mask,double=Double,inst=inst,
                                                transform = VitTCGAToTensor())
s_val = Vit_TCGA_Dataset(v_val_pd, img_loc, z_score=Z_score,sz=SZ, inclued_mask=mask,double=Double,inst=inst,
                                                transform = VitTCGAToTensor())

train_loader = torch.utils.data.DataLoader(s_train, batch_size=Batch_size, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(s_val, batch_size=Batch_size, shuffle=True, num_workers=0)

In [7]:
s_train[0][0].shape

torch.Size([4, 224, 224])

# Moodel

In [8]:
model = timm.create_model('vit_small_patch16_224', pretrained=True, in_chans=channel,num_classes=2 )

In [9]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
print(pl.__version__)

1.8.1


In [10]:
# Setting the seed
pl.seed_everything(42)

Global seed set to 42


42

In [11]:
class JCModule(pl.LightningModule):
    
    def __init__(self, model, optimizer_name, optimizer_hparams):
        """
        Inputs:
              optimizer_name - Name of the optimzer to use. Currently supported: Adam, SGD
              optimizer_hparams - Hyperparameters for the optimizer, as dictiornay. This includes learning rate weight decay, etc.
        """
        super().__init__()
        #Exports the hyperparameters to YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        # Create model
        self.model = model
        # Create loss module
        self.loss_module = nn.CrossEntropyLoss()
        #Example input for visualizing the graph in Tensorboard
        self.example_input_array = torch.zeros((1,channel,224,224), dtype=torch.float32)
        
    def forward(self, imgs):
        # Forward function that is run when visualizing the graph
        return self.model(imgs)
    
    def configure_optimizers(self):
        # We will support Adam or SGD as optimizers.
        if self.hparams.optimizer_name == "Adam":
            #AdamW is Adam with a correct implementation of weight decay
            optimizer = optim.AdamW(
            self.parameters(), **self.hparams.optimizer_hparams)
        elif self.hparams.optimizer_name == "SGD":
            optimizer = optim.SGD(self.parameters(),**self.hparams.optimizer_hparams)
        else:
            assert False, f"Unknown optimizer: \"{self.hparams.optimizer_name}\""
        # We will redudce the learning rate by 0.1 after 100 and 150 epochs
        #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr,total_steps=total_steps)
        return [optimizer]
        
    def training_step(self, batch, batch_idx):
        # "batch" is the output of the training data loader.
        imgs, labels = batch
        labels = labels[3]
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
        acc = (preds.argmax(dim =-1) == labels).float().mean()
        f1score = f1ig0 (preds, labels)
        f1scoreR = f1ig1 (preds, labels)
        #Logs the accuracy per epoch to tensorboard (weighted average over batches)
        self.log('train_acc',acc,on_step=False, on_epoch = True)
        self.log('train_loss', loss)
        self.log('train_F1', f1score)
        self.log('train_F1R', f1scoreR)
        return loss # Return tensor to call ".backward" on
    
    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        labels = labels[3]
        preds = self.model(imgs).argmax(dim =-1)
        acc = (labels == preds).float().mean()
        f1score = f1ig0 (preds, labels)
        f1scoreR = f1ig1 (preds, labels)
        # By default logs it per epoch (weighted average over batches)
        self.log('val_acc', acc)
        self.log('val_F1', f1score)
        self.log('val_F1R', f1scoreR)
        
    def test_step(self, batch, batch_idx):
        imgs, labels = batch
        labels = labels[3]
        preds = self.model(imgs).argmax(dim =-1)
        acc = (labels == preds).float().mean()
        f1score = f1ig0 (preds, labels)
        f1scoreR = f1ig1 (preds, labels)
        # By the default logs it per epoch(weight average over batches), and return it afterwards
        self.log('test_acc', acc)
        self.log('test_F1', f1score)
        self.log('test_F1R', f1scoreR)
        
    def predict_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs).argmax(dim =-1)
        return preds

In [12]:
def train_model(models,save_name, **kwargs):
    """
    Inputs:
        model_name - Name of the model you want to run. It used to look up the class in "model_dict"
        save_name(optitional) - If specified, this name will be used for creating checkpoint and logging directory.
    """
    
    # Create a Pytorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir = os.path.join(CHECKPOINT_PATH, save_name),
                        gpus=1 if str(device)=='cuda:0' else 0,
                        max_epochs = Epoch,
                        callbacks=[ModelCheckpoint(save_on_train_epoch_end='True',mode="max", monitor="val_acc"),
                                  LearningRateMonitor("epoch")])
    trainer.logger._log_graph = True         #If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None
    
    pl.seed_everything(42) # To be reproducable
    model = JCModule(models, **kwargs)
    trainer.fit(model, train_loader, val_loader)
    model = JCModule.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training
        
    # Test best model on validation and test set
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    train_result = trainer.test(model, dataloaders=train_loader, verbose=False)
    result = {"train_acc":train_result[0]["test_acc"], "train_F1":train_result[0]["test_F1"],
              "train_F1R":train_result[0]["test_F1R"],
              "val_acc":val_result[0]["test_acc"],"val_F1":val_result[0]["test_F1"],
             "val_F1R":val_result[0]["test_F1R"]}
    
    return model, result

In [13]:
Vit_model, Vit_results = train_model(model,save_name= save_name, 
                                           optimizer_name="Adam",
                                           optimizer_hparams={"lr": lr})

/uge_mnt/home/satoc/.raiden/nvcr-pytorch-2210/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
Missing logger folder: saved_models/TCGA10fold_fold0/TCGA10fold_fold0/lightning_logs

  | Name        | Type              | Params | In sizes         | Out sizes
---------------------------------------------------------------------------------
0 | model       | VisionTransformer | 21.8 M | [1, 4, 224, 224] | [1, 2]   
1 | loss_module | CrossEntropyLoss  | 0      | ?                | ?        
---------------------------------------------------------------------------------
21.8 M    Train

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

In [14]:
print("Vit Results", Vit_results)

Vit Results {'train_acc': 1.0, 'train_F1': 1.0, 'train_F1R': 1.0, 'val_acc': 0.9333333373069763, 'val_F1': 0.8894623517990112, 'val_F1R': 0.9496920108795166}
